<a href="https://colab.research.google.com/github/Evavanrooijen/AfricanGDP/blob/clean/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install lmfit


     |████████████████████████████████| 266kB 2.8MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 235kB 8.7MB/s 
  Created wheel for lmfit: filename=lmfit-1.0.0-cp36-none-any.whl size=87917 sha256=0b4543c1072d7a79b9284b583d38d99397822f377cfd579db7948b7dd9e5b420
  Stored in directory: /root/.cache/pip/wheels/1f/93/e9/8d3f759e466f5bbfed3ba1d7dc5b3d4389c469a1ec9a2acaee
  Created wheel for asteval: filename=asteval-0.9.18-cp36-none-any.whl size=17381 sha256=b43f67acfcc6c82bc85eccb35b95f7a47f078c109a0da410353cbca072aca1dc
  Stored in directory: /root/.cache/pip/wheels/1c/53/1b/28d929700c9633b1786a4982e6db564ec8326a8930234dcc19
  Created wheel for uncertainties: filename=uncertainties-3.1.2-cp36-none-any.whl size=96455 sha256=94f35e6b9e3552dd859da4161f61707fc3cf87c2cbed62f45d1b16e849c552ab
  Stored in directory: /root/.cache/pip/wheels/d9/d3/0e/5b0b743a8abd50373705427438456da5dc2621891138d7a618
Successfully built lmfit asteval uncertainties


In [0]:
range_N=[50]
range_T=[50]
range_alpha=[2, 5, 10]
range_a=[1, 2, 5, 10]
range_var_eps=[0.5, 1]

for var_eps in range_var_eps:
  for a in range_a:
    for alpha in range_alpha:
      for T in range_T:
        for N in range_N:
          simulation(N, T, alpha, a, var_eps)

In [0]:
def simulation(N, T, alpha, a , var_eps, runs=100):
  """
  Args: DGP specification
  Returns: beaten
  """
 
  T1_size = int((T*a)/(1+a))  
  RMSPE_model = np.zeros(runs)
  RMSPE_IMA = np.zeros(runs)

  for s in range(runs):
    Y = create_DGP(N, T, alpha, var_eps)
    RMPSE = np.zeros([N, T-T2_size])
    RMPSE_IMA = np.zeros([N, T-T2_size])

    for i in range(N):
      for t in range(T-T1_size):
        train = Y[:][:(T1_size+t)] # should be all countries, limited timehorizon
        prediction = fit_and_predict(define_parameters(train, i)) # could also return RMSPE immediately
        truth = Y[:][(T1_size+t)] # 1x1 scalar value
        RMSPE[i][t] = ((truth - prediction) / truth)**2

        # results = IMA_model.fit(trend='nc')
        fc = IMA_model.predict(train[i][T1_size+j+1])

        model_IMA = ARIMA(train, order=(0,1,1))
        model_fit = model_IMA.fit(disp=0) #trend = 'nc', or 'c'
        output = model_fit.forecast()
        prediction_IMA = output[0]
        RMSPE_IMA[i][t] = ((truth - prediction_IMA) / truth)**2
        
        
    RMSPE_model[s] = sqrt(np.mean(RMSPE))
    RMSPE_IMA[s] = sqrt(np.mean(RMSPE_IMA))

  sumBeaten, beaten = compare_models(RMSPE_IMA, RMSPE_model) # over countries or time ?
  exportResults(RMSPE_IMA, RMSPE_model, beaten)
  return print("Our model beats the benchmark {} times!".format(sumBeaten))

In [0]:
def compare_models(RMSPE_IMA, RMSPE_us):
  assert RMSPE_IMA.size == RMSPE_us.size, 'Size mismatch RMSPE arrays'
  beaten = np.zeros(RMSPE_us.size)
  for i in range(RMSPE_IMA.size):
    if (RMSPE_us[i] < RMSPE_IMA[i]):
      beaten[i] = 1
  return np.sum(beaten), beaten

In [0]:
def exportResults(RMSPE_IMA, RMSPE_model, beaten):
  export = pd.DataFrame()

  # averages over sim_runs = 100, N countries
  export['RMSPE IMA'] = RMSPE_IMA
  export['RMSPE model'] = RMSPE_model
  export['beaten'] = beaten
  export['DGP Specification'] = None

  export.to_csv('Simulation Results.csv')
  print('File is exported')